In [1]:
import torch
import torch.nn as nn
from PIL import Image
from torchvision import transforms
import torchvision.transforms.functional as TF
import sys
import time

# -----------------------------------
# 1. 모델 구조 정의 및 가중치 로드
# -----------------------------------
print("1. Loading the fine-tuned model structure and weights...")

# --- 설정값 ---
MODEL_REPO = "facebookresearch/dinov2"
MODEL_NAME = "dinov2_vits14"
NUM_CLASSES = 3 
CLASS_NAMES = ['downy','healthy', 'powdery']
DEVICE = torch.device("cpu")
print(f"--> Using device: {DEVICE}")
# --- 모델 구조 만들기 (학습 코드와 완전히 동일) ---
try:
    # torch.hub를 이용해 DINOv2 모델 구조 로드
    model = torch.hub.load(MODEL_REPO, MODEL_NAME, pretrained=False) # pretrained=False로 설정
    
    # 마지막 분류기(머리)를 우리의 클래스 수에 맞게 교체
    num_features = 384 # ViT-Small의 특징 벡터 크기
    model.head = nn.Linear(num_features, NUM_CLASSES)
    
    # --- 저장된 가중치 불러오기 ---
    model.load_state_dict(torch.load('dinov2_bound_crop.pth', map_location=DEVICE))
    
    model = model.to(DEVICE)
    model.eval()
    print("--> Model loaded successfully.")

except Exception as e:
    print(f"\nERROR: Failed to load the model.")
    print(f"--> Ensure 'dinov2_hub_finetuned_model.pth' is in the same directory.")
    print(f"--> Original Error: {e}")
    sys.exit(1)

1. Loading the fine-tuned model structure and weights...
--> Using device: cpu


Using cache found in C:\Users\51100/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\51100/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\51100/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\51100/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


--> Model loaded successfully.


In [2]:
def resize_by_long_edge(img, size=224):
    """ 긴 변을 기준으로 비율을 유지하며 리사이즈하는 함수 """
    w, h = img.size
    if w > h:
        new_w = size
        new_h = int(h * (size / w))
    else:
        new_h = size
        new_w = int(w * (size / h))
    return img.resize((new_w, new_h))

def pad_to_square(img):
    w, h = img.size
    max_wh = 224 # 최종 크기를 224로 고정
    hp = (max_wh - w) // 2
    vp = (max_wh - h) // 2
    padding = (hp, vp, max_wh - w - hp, max_wh - h - vp) # 정확한 패딩 계산
    return TF.pad(img, padding, 0, 'constant')

In [3]:
# -----------------------------------
# 2. 추론할 이미지 전처리
# -----------------------------------
print("\n2. Preparing the image for inference...")

# 이미지 전처리 파이프라인 (학습 validation 단계와 동일)
preprocess = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


2. Preparing the image for inference...


In [4]:
# -----------------------------------
# 3. 추론 실행 및 결과 확인
# -----------------------------------

def predict(image_path):
    # 시작 시간 기록
    start_time = time.time()
    """지정된 경로의 이미지에 대해 추론을 수행하고 결과를 출력합니다."""
    try:
        # 이미지 열기
        img = Image.open(image_path).convert("RGB")
        
        # 이미지 전처리 및 배치 차원 추가 (1, 3, 224, 224)
        img_tensor = preprocess(img).unsqueeze(0).to(DEVICE)
        
        print(f"\n3. Running inference on '{image_path}'...")
        
        # 기울기 계산 비활성화 (추론 시에는 필요 없음)
        with torch.no_grad():
            # 모델에 이미지 입력
            outputs = model(img_tensor)
            
            # Softmax를 적용하여 출력을 확률로 변환
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            
            # 가장 높은 확률을 가진 클래스 찾기
            top_prob, top_catid = torch.max(probabilities, 0)
            
            # 종료 시간 기록
            end_time = time.time()
            
            # 결과 출력
            # 실행 시간 계산 (밀리초 단위)
            execution_time_ms = (end_time - start_time) * 1000
            print(f"코드 실행 시간: {execution_time_ms:.3f} ms")
            predicted_class = CLASS_NAMES[top_catid]
            confidence = top_prob.item()
            
            print("\n--- Inference Result ---")
            print(f"Predicted Class: {predicted_class}")
            print(f"Confidence: {confidence:.2%}")
            print("------------------------")
            return predicted_class, execution_time_ms
            
    except FileNotFoundError:
        print(f"ERROR: Image file not found at '{image_path}'")
    except Exception as e:
        print(f"ERROR: An error occurred during inference: {e}")

In [6]:
import os
import shutil

def get_test_list(target_dir):
    try:
        all_files = os.listdir(target_dir)
        dir = []
        for file in all_files:
            dir.append(target_dir + file)
        return dir
        
    except FileNotFoundError:
        print(f"❌ 오류: 원본 폴더 '{target_dir}'을(를) 찾을 수 없습니다.")
    except Exception as e:
        print(f"❌ 알 수 없는 오류가 발생했습니다: {e}")

In [7]:
healthy = get_test_list('C:/blooming_AI/crop_dataset/test/healthy/')
powdery = get_test_list('C:/blooming_AI/crop_dataset/test/powdery/')
downy = get_test_list('C:/blooming_AI/crop_dataset/test/downy/')

In [8]:
import pandas as pd
import numpy as np
result = pd.DataFrame(np.zeros((3,3)), ['healthy', 'powdery', 'downy'], ['healthy', 'powdery', 'downy'])

In [9]:
t = 0
# 정상 잎 테스트
for image in healthy:
    cla, ti = predict(image)
    result.at['healthy', cla] += 1
    t += ti
print(f"정상 {result.at['healthy', 'healthy']}개 | 흰가루병 {result.at['healthy', 'powdery']}개 | 노균병 {result.at['healthy','downy']}개 |")
print(f"평균추론시간 = {(t / len(healthy)):.3f}ms")


3. Running inference on 'C:/blooming_AI/crop_dataset/test/healthy/427493_20201001_4_0_0_3_2_13_0_3.jpg'...
코드 실행 시간: 138.784 ms

--- Inference Result ---
Predicted Class: healthy
Confidence: 98.69%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/healthy/427496_20201001_4_0_0_3_2_13_0_6.jpg'...
코드 실행 시간: 146.797 ms

--- Inference Result ---
Predicted Class: healthy
Confidence: 99.99%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/healthy/427544_20201001_4_0_0_3_2_13_0_54.jpg'...
코드 실행 시간: 144.085 ms

--- Inference Result ---
Predicted Class: healthy
Confidence: 99.96%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/healthy/427548_20201001_4_0_0_3_2_13_0_58.jpg'...
코드 실행 시간: 156.001 ms

--- Inference Result ---
Predicted Class: healthy
Confidence: 97.79%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/healthy/427587_20201001_4_0_0_3_2_13_0_97.j

In [10]:
t2 = 0
# 흰가루병 테스트
for image in powdery:
    cla, ti = predict(image)
    result.at['powdery', cla] += 1
    t2 += ti
print(f"정상 {result.at['powdery', 'healthy']}개 | 흰가루병 {result.at['powdery', 'powdery']}개 | 노균병 {result.at['powdery','downy']}개 |")
print(f"평균추론시간 = {(t2 / len(powdery)):.3f}ms")


3. Running inference on 'C:/blooming_AI/crop_dataset/test/powdery/296174_20210910_4_1_a4_3_2_12_1_0.jpg'...
코드 실행 시간: 118.508 ms

--- Inference Result ---
Predicted Class: powdery
Confidence: 99.99%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/powdery/296178_20210910_4_1_a4_3_2_11_0_0.jpg'...
코드 실행 시간: 125.616 ms

--- Inference Result ---
Predicted Class: powdery
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/powdery/296188_20210910_4_1_a4_3_2_12_1_2.jpg'...
코드 실행 시간: 107.805 ms

--- Inference Result ---
Predicted Class: powdery
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/powdery/296189_20210910_4_1_a4_3_2_12_1_3.jpg'...
코드 실행 시간: 100.280 ms

--- Inference Result ---
Predicted Class: powdery
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/powdery/302004_20210913_4_1_a4_3_2_11_

In [11]:
t3 = 0
# 노균병 테스트
for image in downy:
    cla, ti = predict(image)
    result.at['downy',cla] += 1
    t3 += ti
print(f"정상 {result.at['downy','healthy']}개 | 흰가루병 {result.at['downy','powdery']}개 | 노균병 {result.at['downy','downy']}개 |")
print(f"평균추론시간 = {(t3 / len(downy)):.3f}ms")


3. Running inference on 'C:/blooming_AI/crop_dataset/test/downy/470538_20211014_4_1_a3_3_2_13_1_11.jpg'...
코드 실행 시간: 121.506 ms

--- Inference Result ---
Predicted Class: downy
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/downy/470543_20211014_4_1_a3_3_2_13_1_16.jpg'...
코드 실행 시간: 106.715 ms

--- Inference Result ---
Predicted Class: downy
Confidence: 100.00%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/downy/470546_20211014_4_1_a3_3_2_13_1_19.jpg'...
코드 실행 시간: 116.332 ms

--- Inference Result ---
Predicted Class: downy
Confidence: 57.36%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/downy/470547_20211014_4_1_a3_3_2_13_1_20.jpg'...
코드 실행 시간: 121.802 ms

--- Inference Result ---
Predicted Class: downy
Confidence: 99.60%
------------------------

3. Running inference on 'C:/blooming_AI/crop_dataset/test/downy/470550_20211014_4_1_a3_3_2_13_1_23.jpg'...
코드

In [12]:
print(f"최종 평균 추론시간 = {(t+t2+t3) / (len(healthy) + len(powdery) + len(downy)):.3f}ms (연산 장치: 13th Gen Intel(R) Core(TM) i7-13700K)")
# 전체 정확도 (Accuracy) 계산
accuracy = np.diag(result).sum() / result.values.sum()
print(f"## 전체 정확도 (Overall Accuracy)\n- {accuracy:.4f}\n")
classes = ['healthy', 'powdery', 'downy']
# 각 클래스별 지표를 저장할 DataFrame 생성
metrics_df = pd.DataFrame(index=classes, columns=['Precision', 'Recall', 'F1-Score'])

# 각 클래스에 대해 Precision, Recall, F1-Score 계산
for cls in classes:
    tp = result.loc[cls, cls]
    fp = result[cls].sum() - tp
    fn = result.loc[cls].sum() - tp
    
    # Precision (정밀도)
    precision = tp / (tp + fp)
    
    # Recall (재현율)
    recall = tp / (tp + fn)
    
    # F1-Score
    f1_score = 2 * (precision * recall) / (precision + recall)
    
    metrics_df.loc[cls] = [precision, recall, f1_score]

print("## 클래스별 평가 지표\n")
print(metrics_df)

# Macro Average와 Weighted Average 계산
macro_avg = metrics_df.mean()
support = result.sum(axis=1) # 각 클래스의 실제 샘플 수
weighted_avg = (metrics_df.T * support).T.sum() / support.sum()

print("\n## 평균 지표\n")
print(f"- Macro Average:\n{macro_avg.to_string()}\n")
print(f"- Weighted Average:\n{weighted_avg.to_string()}")

최종 평균 추론시간 = 125.136ms (연산 장치: 13th Gen Intel(R) Core(TM) i7-13700K)
## 전체 정확도 (Overall Accuracy)
- 0.9945

## 클래스별 평가 지표

        Precision    Recall  F1-Score
healthy  0.994042   0.99158  0.992809
powdery  0.998084  0.998563  0.998323
downy    0.990391  0.992935  0.991661

## 평균 지표

- Macro Average:
Precision    0.994172
Recall       0.994359
F1-Score     0.994265

- Weighted Average:
Precision    0.994528
Recall       0.994526
F1-Score     0.994526


In [13]:
result

,healthy,powdery,downy
healthy,2002.0,3.0,14.0
powdery,2.0,2084.0,1.0
downy,10.0,1.0,1546.0
